In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from hftbacktest import BacktestAsset, HashMapMarketDepthBacktest
from numba import njit

from data_utils import *

In [2]:
test = demo_convert_okx_csv_for_hftbacktest('btcusdt.csv', 'test.npz', limit=2_500_000)

In [17]:
test

array([(3758096385, 1762560000005000000, 1762560000005000000, 103335.3, 3.0191042e-01, 0, 0, 0.),
       (3758096385, 1762560000005000000, 1762560000005000000, 103335.1, 3.1008500e-03, 0, 0, 0.),
       (3758096385, 1762560000005000000, 1762560000005000000, 103334.8, 1.0000000e-03, 0, 0, 0.),
       ...,
       (3489660929, 1762563931858000000, 1762563931858000000, 103074.1, 2.6567000e-04, 0, 0, 0.),
       (3489660929, 1762563931858000000, 1762563931858000000, 103074.2, 0.0000000e+00, 0, 0, 0.),
       (3489660929, 1762563931858000000, 1762563931858000000, 103075.1, 2.0000000e-03, 0, 0, 0.)],
      shape=(2500000,), dtype={'names': ['ev', 'exch_ts', 'local_ts', 'px', 'qty', 'order_id', 'ival', 'fval'], 'formats': ['<u8', '<i8', '<i8', '<f8', '<f8', '<u8', '<i8', '<f8'], 'offsets': [0, 8, 16, 24, 32, 40, 48, 56], 'itemsize': 64, 'aligned': True})

In [20]:
test['qty']

array([3.0191042e-01, 3.1008500e-03, 1.0000000e-03, ..., 2.6567000e-04,
       0.0000000e+00, 2.0000000e-03], shape=(2500000,))

In [12]:
@njit
def print_depth(hbt, d: int):
    while hbt.elapse(1) == 0:
        print('current_timestamp:', hbt.current_timestamp)

        # Gets the market depth for the first asset, in the same order as when you created the backtest.
        depth = hbt.depth(0)

        # a key of bid_depth or ask_depth is price in ticks.
        # (integer) price_tick = price / tick_size
        i = 0
        for tick_price in range(depth.best_ask_tick, depth.best_ask_tick + 100):
            qty = depth.ask_qty_at_tick(tick_price)
            if qty > 0:
                print(
                    'ask: ',
                    qty,
                    '@',
                    np.round(tick_price * depth.tick_size, 1)
                )
                
                i += 1
                if i == d:
                    break
        i = 0
        for tick_price in range(depth.best_bid_tick, max(depth.best_bid_tick - 100, 0), -1):
            qty = depth.bid_qty_at_tick(tick_price)
            if qty > 0:
                print(
                    'bid: ',
                    qty,
                    '@',
                    np.round(tick_price * depth.tick_size, 1)
                )
            
                i += 1
                if i == d:
                    break
        return True

In [21]:
assert {'ev','exch_ts','local_ts','px','qty','order_id','ival','fval'} \
       <= set(test.dtype.names), "Unexpected dtype – not hftbacktest-ready"

asset = (
    BacktestAsset()
    .data([test])
    .linear_asset(1.0)
    .tick_size(0.1)
    .lot_size(1e-8)
    .constant_latency(0, 0)
)

hbt = HashMapMarketDepthBacktest([asset])

C:\Users\James\AppData\Local\Temp\ipykernel_15000\3062873760.py:10: DeprecationWarning: constant_latency() is deprecated; use constant_order_latency().
  .constant_latency(0, 0)


In [22]:
print_depth(hbt, 10)

current_timestamp: 1762560000005000001
ask:  0.87228425 @ 103335.4
ask:  0.0437 @ 103336.0
ask:  0.15631518 @ 103337.2
ask:  0.00484045 @ 103337.3
ask:  0.0437 @ 103338.0
ask:  0.03286141 @ 103338.1
ask:  0.19 @ 103338.6
ask:  0.0614149 @ 103338.7
ask:  0.0437 @ 103339.0
ask:  0.17568747 @ 103339.4
bid:  0.30191042 @ 103335.3
bid:  0.00310085 @ 103335.1
bid:  0.001 @ 103334.8
bid:  0.020403 @ 103334.3
bid:  1e-05 @ 103334.0
bid:  0.00036271 @ 103333.2
bid:  1.501e-05 @ 103332.5
bid:  1e-05 @ 103332.3
bid:  0.00310063 @ 103330.9
bid:  0.01456209 @ 103330.2


True

In [23]:
hbt.close()

0